In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import pipeline


In [ ]:
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
candidate_labels = ["python generic", "data science", "domain specific", "other"]

In [ ]:
res = classifier('ETF', candidate_labels)

In [ ]:
res

{'sequence': 'ETF',
 'labels': ['other', 'domain specific', 'data science', 'python generic'],
 'scores': [0.46628499031066895,
  0.26926955580711365,
  0.17261715233325958,
  0.09182831645011902]}

In [ ]:
import inspect
import pandas

def is_method_in_package(method_name, package):
    members = inspect.getmembers(package)
    for name, obj in members:
        if name == method_name and inspect.isfunction(obj):
            return True
    return False

method_name_to_check = "plot"  # Replace with the method name you want to check
package_to_inspect = pandas  # Replace with the specific package you want to inspect

if is_method_in_package(method_name_to_check, package_to_inspect):
    print(f"{method_name_to_check} is in {package_to_inspect.__name__}")
else:
    print(f"{method_name_to_check} is not in {package_to_inspect.__name__}")

plot is not in pandas


In [ ]:
!pip install bertopic

In [ ]:
import pandas as pd
import pickle

In [ ]:
QPATH = "Quantlet/token_classification"
# PREPARE WORKING DIRECTORY

import sys
IN_COLAB = 'google.colab' in sys.modules

import os
if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')
else:
  os.chdir('./')

sys.path.append('../src')

import re

In [ ]:
def camel_case_split(str):

    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)

In [ ]:
train = pd.read_csv('../../data/preprocessed/Quantlet/train_df_aut.csv')
val   = pd.read_csv('../../data/preprocessed/Quantlet/val_df_aut.csv')
test  = pd.read_csv('../../data/preprocessed/Quantlet/test_df_aut.csv')

train['subset'] = 'train'
val['subset'] = 'val'
test['subset'] = 'test'


In [ ]:
full = pd.concat([train, val, test], axis=0).reset_index(drop=True)
full['idx'] = full.index

In [ ]:
#full.to_csv('full.csv', index=False)
train = full

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
folder_names = train['folder_name'].str.split('QuantLet', expand=True)[1]
folder_names = folder_names.apply(lambda k: re.findall(r'[a-zA-ZäöüÄÖÜ]+', k))
folder_names = folder_names.apply(lambda k: [' '.join(camel_case_split(word)).lower() if len(camel_case_split(word))>0 else word.lower() for word in k])

train['full_string'] = folder_names.apply(lambda k: ' '.join(k)) + ' ; ' +  train['code_script'] + ' ; ' + train['Description']

In [ ]:
train['full_string_cleaned'] = train['full_string'].apply(lambda k: re.findall(r'[a-zA-ZäöüÄÖÜ]+', k))
train['full_string_cleaned'] = train['full_string_cleaned'].apply(lambda k: [' '.join(camel_case_split(word)).lower() if len(camel_case_split(word))>0 else word.lower() for word in k])
train['full_string_cleaned'] = train['full_string_cleaned'].apply(lambda k: [word for word in k if word not in stopwords])

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(train['full_string_cleaned'].apply(lambda k: ' '.join(k)))

In [ ]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,970,-1_team_home_worldcup_data,"[team, home, worldcup, data, set, cc, index, b...",[sda st gallen cryptocurrencies clustering pyc...
1,0,63,0_sentiment_vader_word_roberta,"[sentiment, vader, word, roberta, words, reddi...",[deda class ss deda class ss reddit sentiment ...
2,1,60,1_outlier_panel_va_jan,"[outlier, panel, va, jan, backtesting, pts, ex...",[sfs qid sf svar block max backtesting book sf...
3,2,57,2_kernel_dax_curve_density,"[kernel, dax, curve, density, fh, imp, news, b...",[mse ms enonpara book mse description ms enonp...
4,3,53,3_eigenvalues_first_eigenvectors_pc,"[eigenvalues, first, eigenvectors, pc, covar, ...",[mva qid mv anpcabanki clear variables rm list...
...,...,...,...,...,...
203,202,11,202_sales_bids_amount_eth,"[sales, bids, amount, eth, sold, punk, top, df...",[deda se cryptopunks transactions analysis exp...
204,203,10,203_corpus_query_articles_lda,"[corpus, query, articles, lda, topic, dtm, fol...",[topic models lda dtm lda import match article...
205,204,10,204_mw_conn_lowertail_adjac,"[mw, conn, lowertail, adjac, dateindex, uppert...",[inet inet totaldire close windows clear varia...
206,205,10,205_tsa_pacf_sbm_fig,"[tsa, pacf, sbm, fig, dpi, ar, acf, inches, sa...",[tsa tsa unit root import numpy np import pand...


In [ ]:
train['full_string_cleaned']

0       [disaster, adaptive, algorithm, close, graph, ...
1       [crix, etf, visualization, simulation, results...
2       [crix, etf, understanding, crypto, et fs, set,...
3       [crix, etf, problem, tickers, names, cryptocur...
4       [crix, etf, understanding, crypto, et fs, set,...
                              ...                        
5532    [quach, symanzik, forsgren, script, calculates...
5533    [quach, symanzik, forsgren, script, creates, h...
5534    [quach, symanzik, forsgren, soul, community, a...
5535    [quach, symanzik, forsgren, script, runs, rand...
5536    [quach, symanzik, forsgren, script, needs, ran...
Name: full_string_cleaned, Length: 5537, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(train['full_string_cleaned'].apply(lambda k: ' '.join(k)))
vectorizer.get_feature_names_out()
print(X.shape)

(5537, 21986)


In [ ]:
word_counts = train['full_string_cleaned'].apply(lambda k: ' '.join(k)).str.split(expand=True).stack().value_counts().reset_index()

word_counts.columns = ['Word', 'Frequency']

word_counts

,Word,Frequency
0,x,43211
1,data,28861
2,n,22049
3,c,21191
4,k,14263
...,...,...
22008,betweeness,1
22009,triples,1
22010,obmit,1
22011,lease,1


In [ ]:
word_counts.query('Frequency>3')

,Word,Frequency
0,x,43211
1,data,28861
2,n,22049
3,c,21191
4,k,14263
...,...,...
13703,pediod,4
13704,datalist,4
13705,dasin,4
13706,fwdata,4


In [ ]:
word_counts['len'] = word_counts.Word.apply(len)

In [ ]:
word_counts.query('len>1 & Frequency>3').to_csv('word_counts.csv', index=False)

In [ ]:
!pip install spacy

In [ ]:
!pip install spacy-entity-linker

In [ ]:
!python -m spacy_entity_linker "download_knowledge_base"

2023-09-12 10:36:03.431402: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'spacy_entity_linker.__main__' found in sys.modules after import of package 'spacy_entity_linker', but prior to execution of 'spacy_entity_linker.__main__'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


In [ ]:
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import spacy  # version 3.0.6'

# initialize language model
nlp = spacy.load("en_core_web_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

In [ ]:
doc = train['full_string_cleaned'].apply(lambda k: ' '.join(k)).iloc[0]

In [ ]:
doc = nlp(doc)

In [ ]:
all_linked_entities = doc._.linkedEntities

In [ ]:
train['full_string_cleaned']

0       [disaster, adaptive, algorithm, close, graph, ...
1       [crix, etf, visualization, simulation, results...
2       [crix, etf, understanding, crypto, et fs, set,...
3       [crix, etf, problem, tickers, names, cryptocur...
4       [crix, etf, understanding, crypto, et fs, set,...
                              ...                        
5532    [quach, symanzik, forsgren, script, calculates...
5533    [quach, symanzik, forsgren, script, creates, h...
5534    [quach, symanzik, forsgren, soul, community, a...
5535    [quach, symanzik, forsgren, script, runs, rand...
5536    [quach, symanzik, forsgren, script, needs, ran...
Name: full_string_cleaned, Length: 5537, dtype: object

In [ ]:
sent._.linkedEntities

<EntityCollection (82 entities):
-https://www.wikidata.org/wiki/Q42213 work                      process or amount (and direction) of energy transfer to an object via the application of forces on i
-https://www.wikidata.org/wiki/Q7300358 read                      computer system call                              
-https://www.wikidata.org/wiki/Q1988072 adaptive algorithm        algorithm that changes its behavior at the time it is run
-https://www.wikidata.org/wiki/Q3839081 disaster                  hazard resulting in an event causing significant physical damage, destruction or death
-https://www.wikidata.org/wiki/Q17514 graffiti                  writing or drawing etched, scratched, scribbled, or sprayed illicitly on a wall or other surface in 
-https://www.wikidata.org/wiki/Q82753 computer file             block of arbitrary information, or resource for storing information, which is available to a compute
-https://www.wikidata.org/wiki/Q1144928 filename                  name of a co

In [ ]:
for sent in doc.sents:
    sent._.linkedEntities.pretty_print()

<EntityElement: https://www.wikidata.org/wiki/Q42213 work                      process or amount (and direction) of energy transfer to an object via the application of forces on i>
<EntityElement: https://www.wikidata.org/wiki/Q7300358 read                      computer system call                              >
<EntityElement: https://www.wikidata.org/wiki/Q1988072 adaptive algorithm        algorithm that changes its behavior at the time it is run>
<EntityElement: https://www.wikidata.org/wiki/Q3839081 disaster                  hazard resulting in an event causing significant physical damage, destruction or death>
<EntityElement: https://www.wikidata.org/wiki/Q17514 graffiti                  writing or drawing etched, scratched, scribbled, or sprayed illicitly on a wall or other surface in >
<EntityElement: https://www.wikidata.org/wiki/Q82753 computer file             block of arbitrary information, or resource for storing information, which is available to a compute>
<EntityElement:

In [47]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
#model = AutoModel.from_pretrained("microsoft/codebert-base")
nl_tokens=tokenizer.tokenize("""import spacy  # version 3.0.6'

# initialize language model
nlp = spacy.load("en_core_web_md")

# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)""")